In [102]:
#Here we import all the necessary dependencies
import numpy as np
import matplotlib.pyplot as plt
import time
from IPython.display import clear_output
from amuse.lab import units, constants
from amuse.ext.orbital_elements import new_binary_from_orbital_elements
from amuse.lab import Particles

In [103]:
G = 6.67259*10**-11
AU = 149597871000

In [104]:
def cross(a, b):
    c = (a[1]*b[2] - a[2]*b[1],
         a[2]*b[0] - a[0]*b[2],
         a[0]*b[1] - a[1]*b[0])

    return c

In [105]:
#Here we generate a basic solarsystem, with only the gas giants
from amuse.ext.solarsystem import new_solar_system

def create_system():
    
    system = new_solar_system()
    system = system[system.mass > 10**-5 | units.MSun]
    system.move_to_center()
    return system
    
    
basic_giants_system = create_system()

|S11


In [108]:
#Here we add the Oort cloud objects, according to a chosen distribution
def random_position(): #There is probably some AMUSE function for this
    semi_major_axis = np.random.uniform(4, 40) 
    inclination = np.random.uniform(-5, 5) * (2*np.pi/360) #Tranforming to rad
    
    z = (np.sqrt((np.tan(inclination)**2)/(1+np.tan(inclination)**2))*semi_major_axis)
    remainder = (np.sqrt(semi_major_axis**2-z**2))
    y = np.random.uniform(0, remainder)
    x = np.sqrt(remainder**2-y**2)
    position = (x, y, z)
    return position, inclination
    
def random_velocity(position, inclination, system): #There is probably some AMUSE function for this
    distance = np.sqrt(position[0]**2+position[1]**2+position[2]**2)
    normal_plane = np.sqrt((distance**2)/(1+(1/(np.tan(inclination)**2))))
    
    z_normal = (normal_plane/np.tan(inclination))
    y_normal = (np.sqrt(distance**2-z_normal**2)*position[1]/np.sqrt(position[0]**2+position[1]**2))
    x_normal = (np.sqrt(distance**2-z_normal**2)*position[0]/np.sqrt(position[0]**2+position[1]**2))
    
    normal_vector = (x_normal, y_normal, z_normal)

    normalised_normal = (normal_vector/(np.sqrt(normal_vector[0]**2+normal_vector[1]**2+normal_vector[2]**2)))
    normalised_position = (position/(np.sqrt(position[0]**2+position[1]**2+position[2]**2)))
    
    normalised_velocity = cross(normalised_normal, normalised_position)
    velocity_length = np.sqrt(G*system.mass.sum().value_in(units.kg)/(distance*AU))
    
    vx = normalised_velocity[0]*velocity_length
    vy = normalised_velocity[1]*velocity_length
    vz = normalised_velocity[2]*velocity_length
    
    return (vx/1000., vy/1000., vz/1000.)
    
def add_oort_objects(system, N_objects):
    
    for i in range(N_objects):
        oort = Particles(1)
        oort.name = "OORT_" + str(i)
        oort.mass = 0.0 | units.MSun
        position = random_position()
        velocity = random_velocity(position[0], position[1], system)
        oort.position = (position[0][0], position[0][1], position[0][2]) | units.AU#Take eccentricity + inclination + semi-major axis according to some dist.
        oort.velocity = (velocity[0], velocity[1], velocity[2])  | units.kms

        system.add_particle(oort)
    return system

complete_system = add_oort_objects(basic_giants_system, N_objects = 10**4)

In [109]:
print(complete_system)

                 key         mass         name       radius           vx           vy           vz            x            y            z
                   -         MSun         none         RSun       km / s       km / s       km / s           AU           AU           AU
====================  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========
10140836739351626027    1.000e+00          SUN    1.000e+00   -1.835e-03   -1.551e-02    1.133e-04   -8.352e-03    1.921e-03    2.145e-04
 3266837975900762502    9.548e-04      JUPITER    1.619e-01    2.873e+00    1.331e+01   -1.194e-01    4.833e+00   -1.158e+00   -1.034e-01
17835025470818215274    2.859e-04       SATURN    1.342e-01   -4.794e+00    8.639e+00    4.001e-02    8.335e+00    4.127e+00   -4.033e-01
17449353621925607954    4.366e-05       URANUS    5.834e-02    5.131e+00    4.103e+00   -5.124e-02    1.289e+01   -1.511e+01   -2.231e-01
10202460868426967333    5.151e-05 

In [19]:
#Here we generate a galactic potential 

class MilkyWay_galaxy(object):
    def __init__(self, 
                 Mb=1.40592e10| units.MSun,
                 Md=8.5608e10| units.MSun,
                 Mh=1.07068e11 | units.MSun):
        self.Mb= Mb
        self.Md= Md
        self.Mh= Mh

    def get_potential_at_point(self,eps,x,y,z):
        r=(x**2+y**2+z**2)**0.5
        R= (x**2+y**2)**0.5
        # buldge
        b1= 0.3873 |units.kpc
        pot_bulge= -constants.G*self.Mb/(r**2+b1**2)**0.5 
        # disk
        a2= 5.31 |units.kpc
        b2= 0.25 |units.kpc
        pot_disk = -constants.G*self.Md/(R**2 + (a2+ (z**2+ b2**2)**0.5 )**2 )**0.5
        #halo
        a3= 12.0 |units.kpc
        cut_off=100 |units.kpc
        d1= r/a3
        c=1+ (cut_off/a3)**1.02
        pot_halo= -constants.G*(self.Mh/a3)*d1**1.02/(1+ d1**1.02) \
                  - (constants.G*self.Mh/(1.02*a3))\
                      * (-1.02/c +numpy.log(c) + 1.02/(1+d1**1.02) \
                           - numpy.log(1.0 +d1**1.02) )
        return 2*(pot_bulge+pot_disk+ pot_halo) 
                # multiply by 2 because it is a rigid potential
    
    def get_gravity_at_point(self, eps, x,y,z): 
        r= (x**2+y**2+z**2)**0.5
        R= (x**2+y**2)**0.5
        #bulge
        b1= 0.3873 |units.kpc
        force_bulge= -constants.G*self.Mb/(r**2+b1**2)**1.5 
        #disk
        a2= 5.31 |units.kpc
        b2= 0.25 |units.kpc
        d= a2+ (z**2+ b2**2)**0.5
        force_disk=-constants.G*self.Md/(R**2+ d**2 )**1.5
        #halo
        a3= 12.0 |units.kpc
        d1= r/a3
        force_halo= -constants.G*self.Mh*d1**0.02/(a3**2*(1+d1**1.02))
       
        ax= force_bulge*x + force_disk*x  + force_halo*x/r
        ay= force_bulge*y + force_disk*y  + force_halo*y/r
        az= force_bulge*z + force_disk*d*z/(z**2 + b2**2)**0.5 + force_halo*z/r 

        return ax,ay,az
    
MW_potential = MilkyWay_galaxy()

In [ ]:
#Here we create the conditions for the migration of the planets

def planet_migration():
    
    velocities = 
    return velocities

final_system = complete_system.add(velocities)
final_system.move_to_center()

In [ ]:
#Here we perform the conversion for the system

final_converter=nbody_system.nbody_to_si(final_system.mass.sum(), #CHECK IF THIS IS CORRECT
                                   final_system.position.length())

In [ ]:
#Here we evolve the system, also plotting the spatial evolution of the objects
from amuse.community.ph4.interface import ph4

def evolver(particle_system, potential, converter, end_time=4*10**3, time_step=0.1):
    
    gravity_code = ph4(converter)
    gravity_code.particles.add_particles(particle_system)
    ch_g2l = gravity_code.particles.new_channel_to(particle_system)
    
    gravity_bridge = 0
    gravity_bridge = bridge.Bridge(use_threading=False)
    gravity_bridge.add_system(gravity_code, (potential,) )
    gravity_bridge.timestep = time_step|units.Myr

    times = numpy.arange(0., end_time, time_step) | units.Myr
    for time in times:
        gravity_bridge.evolve_model(time)
        ch_g2l.copy()
    gravity.stop()
    
    
    
evolved_system = evolver(final_system, MW_potential, final_converter end_time=, time_step=)